Pour un bon fonctionnement, il est impératif d'utiliser une version d'opencv-contrib inférieure à 4.7, comme par exemple opencv-contrib-python-4.6.0.66.

Sinon, le Charuco board ne sera pas détecté en raison de modifications et corrections en cours dans l'API d'OpenCV.

In [19]:
import numpy as np
import cv2
from cv2 import aruco
import glob

Importer les images des caméras

In [20]:
images_calibration_b1 = glob.glob('C:/Users/petitq/Downloads/B1/Calibration_B1/Intrinsic/*.tiff')
images_calibration_b2 = glob.glob('C:/Users/petitq/Downloads/B2/Calibration_B2/Intrinsic/*.tiff')
images_calibration_b3 = glob.glob('C:/Users/petitq/Downloads/B3/Calibration_B3/Intrinsic/*.tiff')

images_b1 = glob.glob('C:/Users/petitq/Downloads/B1/GM8_B1/*.tiff')
images_b2 = glob.glob('C:/Users/petitq/Downloads/B2/GM8_B2/*.tiff')
images_b3 = glob.glob('C:/Users/petitq/Downloads/B3/GM8_B3/*.tiff')

Calibration intrinsèque qui renvoie les coefficients

In [21]:
def calibrate_camera(images):
    # ChAruco board variables
    CHARUCOBOARD_ROWCOUNT = 6
    CHARUCOBOARD_COLCOUNT = 10
    ARUCO_DICT = aruco.Dictionary_get(aruco.DICT_5X5_1000)
    # Create constants to be passed into OpenCV and Aruco methods
    CHARUCO_BOARD = aruco.CharucoBoard_create(
        squaresX=CHARUCOBOARD_COLCOUNT,
        squaresY=CHARUCOBOARD_ROWCOUNT,
        squareLength=0.04,
        markerLength=0.02,
        dictionary=ARUCO_DICT
    )
    # Create the arrays and variables we'll use to store info like corners and IDs from images processed
    corners_all = [] # Corners discovered in all images processed
    ids_all = [] # Aruco ids corresponding to corners discovered
    image_size = None # Determined at runtime

    # Loop through images glob'ed
    for iname in images:
        # Open the image
        img = cv2.imread(iname)
        # Grayscale the image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find aruco markers in the query image
        corners, ids, _ = aruco.detectMarkers(
            image=gray,
            dictionary=ARUCO_DICT
        )

        # Outline the aruco markers found in our query image
        img = aruco.drawDetectedMarkers(
            image=img, 
            corners=corners
        )

        # Get charuco corners and ids from detected aruco markers
        response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=gray,
            board=CHARUCO_BOARD
        )

        # If a Charuco board was found, let's collect image/corner points
        # Requiring at least 20 squares
        if response > 20:
            # Add these corners and ids to our calibration arrays
            corners_all.append(charuco_corners)
            ids_all.append(charuco_ids)

            # Draw the Charuco board we've detected to show our calibrator the board was properly detected
            img = aruco.drawDetectedCornersCharuco(
                    image=img,
                    charucoCorners=charuco_corners,
                    charucoIds=charuco_ids)
            
            # If our image size is unknown, set it now
            if not image_size:
                image_size = gray.shape[::-1]
                
            # Reproportion the image, maxing width or height at 1000
            proportion = max(img.shape) / 1000.0
            img = cv2.resize(img, (int(img.shape[1]/proportion), int(img.shape[0]/proportion)))
            # Pause to display each image, waiting for key press
            cv2.imshow('Charuco board', img)
            cv2.waitKey(0)
        else:
            print("Not able to detect a charuco board in image: {}".format(iname))

    # Destroy any open CV windows
    cv2.destroyAllWindows()

    # Make sure at least one image was found
    if len(images) < 1:
        # Calibration failed because there were no images, warn the user
        print("Calibration was unsuccessful. No images of charucoboards were found. Add images of charucoboards and use or alter the naming conventions used in this file.")
        # Exit for failure
        exit()

    # Make sure we were able to calibrate on at least one charucoboard by checking
    # if we ever determined the image size
    if not image_size:
        # Calibration failed because we didn't see any charucoboards of the PatternSize used
        print("Calibration was unsuccessful. We couldn't detect charucoboards in any of the images supplied. Try changing the patternSize passed into Charucoboard_create(), or try different pictures of charucoboards.")
        # Exit for failure
        exit()

    # Now that we've seen all of our images, perform the camera calibration
    # based on the set of points we've discovered
    ret, mtx, dist, rvecs, tvecs = aruco.calibrateCameraCharuco(
        charucoCorners=corners_all,
        charucoIds=ids_all,
        board=CHARUCO_BOARD,
        imageSize=image_size,
        cameraMatrix=None,
        distCoeffs=None)
    return ret, mtx, dist, rvecs, tvecs

matrix_b1 = calibrate_camera(images_calibration_b1)
matrix_b2 = calibrate_camera(images_calibration_b2)
matrix_b3 = calibrate_camera(images_calibration_b3)
        

Corriger la distorsion des images choisies

In [22]:
def undistort(images, mtx, dist):
    # Loop through images glob'ed
    all_undistorted_images = []
    for iname in images:
        # Open the image
        img = cv2.imread(iname)
        img = cv2.rotate(img,cv2.ROTATE_90_COUNTERCLOCKWISE)
        # Grayscale the image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Undistort the image
        undistorted_img = cv2.undistort(gray, mtx, dist)
        all_undistorted_images.append(undistorted_img)
        
    return all_undistorted_images

undistorted_images_b1 = undistort(images_b1, matrix_b1[1], matrix_b1[2])
undistorted_images_b2 = undistort(images_b2, matrix_b2[1], matrix_b2[2])
undistorted_images_b3 = undistort(images_b3, matrix_b3[1], matrix_b3[2])

In [23]:
img = undistorted_images_b3[0]

## 2.1 Create MSER object
mser = cv2.MSER_create()

## 2.2 Detect MSER regions
regions,_ = mser.detectRegions(img)

## 2.3 Draw MSER regions
# cv2.convexHull finds the convex hull of a 2D point set
# save the list of numpy array of contours a variable ‘hulls’
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]

# create a copy of the original image
clone = img.copy()
clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

# draw contours on the image.
cv2.polylines(clone, hulls, 1, (0, 255, 0))

# show and save the image
cv2.namedWindow('all MSER', cv2.WINDOW_NORMAL)
cv2.imshow('all MSER', clone) # show the image
cv2.imwrite('all MSER.jpg',clone) # save the image
cv2.waitKey()
cv2.destroyAllWindows()

def circularity_check(img, regions, hulls):

    ## Circularity Check
    # create two empty lists to store interesting regions
    Regions = []
    Contours = []

    # iterate over regions and contours
    # zip() returns an iterator of tuples based on the iterable objects
    # enumerate() method adds a counter to an iterable and
    # returns it in a form of enumerating object
    for index, (region, cnt) in enumerate(zip(regions, hulls)):
        area = cv2. contourArea(cnt) # computes the area of the contours
        perimeter = cv2.arcLength(cnt,True) # computes contours perimeter
        circularity = 4*np.pi*area/perimeter**2 # circularity index
        if circularity > 0.93:
            if(len(cnt) < 14):
                Regions.append(region) # interesting regions
                Contours.append(cnt) # interesting contours

    ## Plot circular MSER

    # create a copy of the original image and convert it in gray image
    clone = img.copy()
    clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # draw MSER
    cv2.polylines(clone, Contours, 1, (0, 255, 0))
    cv2.namedWindow('circular MSER', cv2.WINDOW_NORMAL)
    cv2.imshow('circular MSER', clone)
    cv2.imwrite('circular MSER.jpg',clone) # write the image
     # close the image
    cv2.waitKey()
    cv2.destroyAllWindows()

    return Regions, Contours, clone


Regions_circ, Contours_circ, clone_circ = circularity_check(img, regions, hulls)

In [24]:
def intensity_check(img, regions, hulls):
    ## Check on the pixel intensity

    # create three empty lists to store intereseting regions
    Regions = []
    Contours = []
    Centers = []

    # iterate over regions and contours
    for index, (region, cnt) in enumerate(zip(regions, hulls)):
        M = cv2.moments(cnt) # calculate the image moments
        center = int(M["m10"]/M["m00"]),int(M["m01"]/M["m00"]) # centroid (Cx, Cy)
        bright = img[center[1],center[0]] # brightness of the center (expected white)
        bright_mean = np.mean([img[p[1],p[0]] for p in region]) # mean brightness within a region
        if bright >= 0:
            Regions.append(region) # interesting regions
            Contours.append(cnt) # interesting contours
            Centers.append(center) # interesting centroids

    ## Plot selected MSER

    # create a copy of the original image
    clone = img.copy()
    clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # draw and save MSER
    cv2.polylines(clone, Contours, 1, (0, 255, 0))


    cv2.namedWindow('selected MSER from intensity', cv2.WINDOW_NORMAL)
    cv2.imshow('selected MSER from intensity',clone)
    cv2.imwrite('selected MSER from intensity.jpg',clone)
    # close the image
    cv2.waitKey()
    cv2.destroyAllWindows()

    return Regions, Contours, Centers, clone

Regions_int, Contours_int, Centers_int, clone_int = intensity_check(img, Regions_circ, Contours_circ)

In [25]:
def duplicated_check(img, regions, cnt, centers):
    Box = []
    Box = [cv2.boundingRect(i) for i in cnt] # bounding box
    B = np.array(Box) # convert into array
    # create a list with 0 -> False in Boolean data type
    remove = np.zeros(B.shape[0], dtype=bool) # 0 is equal to False

    atol = 5 # tolerance
    # iterate over all boxes
    for i in range(B.shape[0]):

        # check if a box is similar to the others within a fixed
        # tolerance
        # np.isclose returns a boolean array (True or False)
        # where two arrays are element-wise equal within a tolerance
        similar = np.isclose(B[i, :], B[(i + 1):, :], atol = atol)
        # check if a box is similar to another on both sides

        # np.all Test whether all array elements along a given axis
        # evaluate to True
        equals = np.all(similar, axis=1)
        # Logical OR: True if either of the operands is true
        remove[(i + 1):] = np.logical_or(remove[(i + 1):], equals)

    # np.where return elements chosen from x or y depending on condition
    # Logical NOT: True if operand is false
    index = np.where(np.logical_not(remove))
    Bbox = B[np.logical_not(remove)]
    # create three empty lists to store intereseting regions
    Regions = []
    Contours = []
    Centers = []

    Contours = [cnt[p] for p in index[0]]
    Regions = [regions[p] for p in index[0]]
    Centers = [centers[p] for p in index[0]]

    ## Plot selected MSER

    # create a copy of the original image
    clone = img.copy()
    clone = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

    # draw MSER
    cv2.polylines(clone, Contours, 1, (0, 255, 0))
    cv2.namedWindow('duplicated MSER suppression', cv2.WINDOW_NORMAL)
    cv2.imshow('duplicated MSER suppression',clone)
    cv2.imwrite('duplicated MSER suppression.jpg',clone)

    # close the image
    cv2.waitKey()
    cv2.destroyAllWindows()
    return Regions, Contours, Centers, Bbox, clone

Regions, Contours, Centers, Bbox, clone_sel = duplicated_check(img,Regions_int,Contours_int,Centers_int)

In [26]:
## 3.4 Evaluate mean centroid
result_centroid = []
temp = []
for ind in Bbox:
    temp = []
    corner_min = np.array((ind[0],ind[1]))
    corner_max = np.array((ind[0]+ind[2],ind[1]+ind[3]))

    for centroid in Centers:
        if ((np.array(centroid)<corner_max).all() and (np.array(centroid)>corner_min).all()):
            temp.append(centroid)
    result_centroid.append(tuple(np.mean(temp,axis=0,dtype = int)))

result_centroid = list(set(result_centroid))
result_centroid = sorted(result_centroid, key=lambda x: x[1], reverse=True)

In [27]:
## 4.1 Label selected MSER
# create a copy of the image and convert it in gray image
clone1 = img.copy()
clone1 = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

## set text font
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
color = (0, 0, 255)
thickness = 2
nombre=0

## put number on the image
for i,c in enumerate(result_centroid):
    clone1 = cv2.rectangle(clone1, (c[0]-10,c[1]-10), (c[0]+10,c[1]+10), (0, 255, 255), thickness)
    nombre+=1
    number = str(i+1)
    cv2.putText(clone1,number,c,font,fontScale,color,thickness)
    
## 4.2 Draw contours
print (nombre)
cv2.polylines(clone1, Contours, 1, (0, 255, 0))
cv2.namedWindow('selected MSER', cv2.WINDOW_NORMAL)
cv2.imshow('selected MSER',clone1) # show the image
cv2.imwrite('selected MSER.jpg',clone1) # save the image

# close the image
cv2.waitKey()
cv2.destroyAllWindows()

46


In [16]:
images_test1 = glob.glob('C:/Users/petitq/Downloads/B1/Calibration_B1/Image__2024-01-17__16-36-39.tiff')
images_test2 = glob.glob('C:/Users/petitq/Downloads/B1/Calibration_B1/Image__2024-01-17__11-40-42.tiff')
images_test3 = glob.glob('C:/Users/petitq/Downloads/B1/Calibration_B1/Image__2024-01-17__20-47-47.tiff')

# TEST EXTRIN

In [66]:
# loop on images
# read image and convert into gray scale



nb_c=5
nb_r=9
d=1

threedpoints = [] # 3d point in real world space (object points)
twodpoints = [] # 2d points in image plane (image points)
## Termination criteria for corner cv.cornerSubPix:
# stop the iteration when specified
# the process of corner position refinement stops either after
# criteria.maxCount iterations or when the corner position
# moves by less than criteria.epsilon on some iteration
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

#image_test = 'C:/Users/petitq/Downloads/B1/Calibration_B1/Image__2024-01-17__16-36-39.tiff'
#image_test = 'C:/Users/petitq/Downloads/B1/Calibration_B1/Image__2024-01-17__16-36-39.tiff'
image_test = 'C:/Users/petitq/Downloads/B3/Calibration_B3/Image__2024-01-17__20-47-47.tiff'

img = cv2.imread(image_test)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # convert to gray

cv2.imshow("test", gray)
cv2.waitKey()
cv2.destroyAllWindows()

# Find the chess board corners
# cv2.findChessboardCorners(image, patternSize, flags)
# patternSize -> num columns, rows

ret, corners = cv2.findChessboardCorners(gray, (nb_c,nb_r), None)
# If found, add object points, image points (after refining them)

print(corners)

if ret == True:

    threedpoints.append(grid)
    # Refines the corner locations
    # for given 2d points
    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
    twodpoints.append(corners2)
    # Draw and display the corners
    image = cv2.drawChessboardCorners(img, (nb_c, nb_r), corners2, ret)
    cv2.imshow("test",image)
    cv2.waitKey()
    cv2.destroyAllWindows()
    




[[[113.44879  540.293   ]]

 [[127.37791  540.1797  ]]

 [[141.13211  540.2741  ]]

 [[154.90271  540.2263  ]]

 [[168.61108  540.273   ]]

 [[114.00877  553.4086  ]]

 [[127.74109  553.45355 ]]

 [[141.50922  553.4524  ]]

 [[155.29066  553.4868  ]]

 [[169.00954  553.44617 ]]

 [[114.43564  566.6425  ]]

 [[128.27075  566.49365 ]]

 [[141.92433  566.6174  ]]

 [[155.58997  566.52747 ]]

 [[169.33124  566.6266  ]]

 [[114.87655  579.60046 ]]

 [[128.58865  579.725   ]]

 [[142.38643  579.62775 ]]

 [[156.13353  579.72504 ]]

 [[169.72617  579.6432  ]]

 [[115.39375  592.9459  ]]

 [[129.14565  592.8084  ]]

 [[142.75761  592.92505 ]]

 [[156.45807  592.8239  ]]

 [[170.19603  592.89124 ]]

 [[115.955765 605.92236 ]]

 [[129.56393  605.9599  ]]

 [[143.2491   605.8965  ]]

 [[156.92636  605.9811  ]]

 [[170.4761   605.90564 ]]

 [[116.39336  618.9693  ]]

 [[130.08556  618.8662  ]]

 [[143.62372  618.97845 ]]

 [[157.32535  618.84875 ]]

 [[170.98825  618.9255  ]]

 [[116.92608  631.79